In [2]:
# -*- coding:utf-8 -*-
# !/usr/bin/python
# Author: Selvaria

# 模型构建和设计

import tensorflow as tf
import numpy as np
print(tf.__version__)

2.1.0


In [2]:
#自定义MLP模型结构

class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = self.dense1(x)    
        output = self.dense2(x)     
        return output
    
X = tf.random.uniform((2,20))
net = MLP()
net(X)

<tf.Tensor: id=62, shape=(2, 10), dtype=float32, numpy=
array([[ 0.14354907,  0.05338682, -0.40425608, -0.2587788 , -0.08983592,
         0.24334475,  0.08793818, -0.5242768 , -0.06720618,  0.17471312],
       [-0.01480664,  0.16989782, -0.4945268 , -0.17466378,  0.14202915,
        -0.02115927, -0.0175516 , -0.611707  , -0.027931  ,  0.10718849]],
      dtype=float32)>

In [3]:
# 正常的keras实现：也可以用add函数

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

model(X)

<tf.Tensor: id=117, shape=(2, 10), dtype=float32, numpy=
array([[-0.0521878 ,  0.0668958 , -0.06255163,  0.03062708, -0.14593492,
         0.10147388, -0.04072712, -0.11214042, -0.02359716, -0.13605078],
       [-0.2278136 , -0.04991286,  0.08795225,  0.17137462, -0.02623325,
         0.07074243, -0.05856334, -0.07146536,  0.12810664, -0.07756173]],
      dtype=float32)>

In [4]:
# 更灵活一些

class FancyMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.rand_weight = tf.constant(
            tf.random.uniform((20,20)))
        self.dense = tf.keras.layers.Dense(units=20, activation=tf.nn.relu)

    def call(self, inputs):         
        x = self.flatten(inputs)   
        x = tf.nn.relu(tf.matmul(x, self.rand_weight) + 1)
        x = self.dense(x)    
        while tf.norm(x) > 1:
            x /= 2
        if tf.norm(x) < 0.8:
            x *= 10
        return tf.reduce_sum(x)

In [5]:
# 定义后可直接通过add函数调用

class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Flatten())
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)


    def call(self, inputs):         
        return self.dense(self.net(inputs))

net = tf.keras.Sequential()
net.add(NestMLP())
net.add(tf.keras.layers.Dense(20))
net.add(FancyMLP())

net(X)

<tf.Tensor: id=311, shape=(), dtype=float32, numpy=23.746525>

In [3]:
# 模型的权重

net = tf.keras.models.Sequential()
net.add(tf.keras.layers.Flatten())
net.add(tf.keras.layers.Dense(256,activation=tf.nn.relu))
net.add(tf.keras.layers.Dense(10))

X = tf.random.uniform((2,20))
Y = net(X)
Y

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.02536002,  0.21001825,  0.2942347 ,  0.37226382, -0.27594352,
        -0.1072491 , -0.01887954, -0.16058698, -0.0123478 ,  0.0620673 ],
       [-0.00090306,  0.30016986,  0.2668451 ,  0.20013687, -0.45570928,
        -0.09548484, -0.31233752, -0.21917205,  0.00874135,  0.05590382]],
      dtype=float32)>

In [4]:
net.weights[0], type(net.weights[0])

(<tf.Variable 'sequential/dense/kernel:0' shape=(20, 256) dtype=float32, numpy=
 array([[ 0.04821877,  0.01971839, -0.1322133 , ..., -0.02457005,
         -0.0513386 , -0.04596808],
        [-0.08145669,  0.10399023,  0.00986847, ...,  0.09806436,
         -0.08687238,  0.09704998],
        [ 0.00680166,  0.14200485,  0.11452475, ..., -0.00169985,
          0.03269209,  0.07975852],
        ...,
        [ 0.14109606,  0.10782537, -0.04107142, ...,  0.08147974,
          0.08208396,  0.02260047],
        [-0.08611073,  0.12286136,  0.01139548, ...,  0.13120255,
          0.10408655,  0.0368717 ],
        [ 0.09235047, -0.03619536, -0.146383  , ..., -0.13490865,
          0.09247346, -0.03999708]], dtype=float32)>,
 tensorflow.python.ops.resource_variable_ops.ResourceVariable)

In [5]:
# 初始化 设为均值为0、标准差为0.01的正态分布随机数(代码里似乎并非这样），并依然将偏差参数清零

class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.d1 = tf.keras.layers.Dense(
            units=10,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )
        self.d2 = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.ones_initializer(),
            bias_initializer=tf.ones_initializer()
        )

    def call(self, input):
        x = self.d1(input)
        output = self.d2(x)
        return output

net = Linear()
net(X)
net.get_weights()

[array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

In [7]:
net.weights

[<tf.Variable 'linear/dense_2/kernel:0' shape=(20, 10) dtype=float32, numpy=
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0.

In [9]:
# 自定义初始化，感觉没啥区别

def my_init():
    return tf.keras.initializers.Ones()

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(64, kernel_initializer=my_init()))

Y = model(X)
model.weights

[<tf.Variable 'sequential_2/dense_5/kernel:0' shape=(20, 64) dtype=float32, numpy=
 array([[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'sequential_2/dense_5/bias:0' shape=(64,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]